# MusixMatch Scraper

In [1]:
#requests
import requests
import urllib
import urllib.request
from urllib.request import urlopen
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
from datetime import datetime
import requests
from urllib.request import urlopen, Request

#data, strucuture and maths
import pandas as pd
import numpy as np
import math
import json
import string
from  more_itertools import unique_everseen
import random

#progress,performance and management
from tqdm import tqdm_notebook
import threading
import os
import ssl
from IPython.display import clear_output
import platform
import os

#time
from time import sleep
import time

In [2]:
class LyricScraper():

    def __init__(self,artist_url):

        self.artist_url = artist_url
        
        self.artist = artist_url.split('artist/')[-1]
         
        self.song_l = []
                        
    def _get_html(self,url):

        headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

        req = Request(url, headers=headers)

        return BeautifulSoup(urlopen(req).read(), 'html.parser')

    def _multithreadCompile(self,thread_count,iteration_list,func):

        jobs = []

        #distribute iteration list to batches and append to jobs list
        batches = [i.tolist() for i in np.array_split(iteration_list,thread_count)]

        for i in range(len(batches)):

            jobs.append(threading.Thread(target=func,args=[batches[i]]))
            
        return jobs

    def _multithreadExecute(self,jobs):

        # Start the threads
        for j in jobs:
    
            j.start()

        # Ensure all of the threads have finished
        for j in jobs:
            j.join()
        return
    
    def _getpageUrls(self,url):
     
        html = self._get_html(url)
        
        songs = html.find_all('h2',{'class':'media-card-title'})
        
        song_urls = ['https://www.musixmatch.com'+i.find('a')['href'] for i in songs]
        
        return [i for i in song_urls if 'album' not in i]


    def _getLyrics(self,song_url):
    
        html = self._get_html(song_url)

        element = html.find_all('span',{'class':'lyrics__content__ok'})

        element_loop = len(element)

        song_lyrics = []

        song_lyrics_raw = [element[i].text.split('\n') for i in range(element_loop)]

        song_lyrics_raw = [i for sublist in song_lyrics_raw for i in sublist]

        song_lyrics.extend(list(dict.fromkeys(song_lyrics_raw)))

        song_lyrics = ' '.join([i for i in song_lyrics if len(i) >0])

        return song_lyrics
    
    def _getAllpageUrls(self,target=45):
        
        loops = int(target/15)
        
        artist_urls = [self.artist_url+'/'+str(i+1) for i in range(loops)]
        
        all_song_urls = []
        
        for i in artist_urls:
            
            all_song_urls.extend(self._getpageUrls(i))
        
        return all_song_urls
              
    def _extractData(self,all_song_urls):
    
        for i in tqdm_notebook(range(len(all_song_urls))):
            
            try:
            
                song_lyrics = self._getLyrics(all_song_urls[i])
                
                song_title = all_song_urls[i].split('/')[-1]

                song_df = pd.DataFrame([(self.artist,song_title,song_lyrics)],columns=['artist','song','lyrics'])
                
                self.song_l.append(song_df)
            
            except:
                pass

        return 
    
    def Run(self,target):
       
        self.all_song_urls = self._getAllpageUrls(target)

        self._multithreadExecute(self._multithreadCompile(5,self.all_song_urls,self._extractData))

        try:
            df_final = pd.concat([i for i in self.song_l])

            df_final.reset_index(drop=True,inplace=True)

            return df_final[df_final.lyrics.str.len() > 0]

        except:
            pass
            return
     

In [9]:
artist = ['https://www.musixmatch.com/artist/Kylie-Minogue',
    
'https://www.musixmatch.com/artist/Snoop-Dogg',
    
'https://www.musixmatch.com/artist/Cannibal-Corpse',
    
'https://www.musixmatch.com/artist/Kanye-West',
    
'https://www.musixmatch.com/artist/Nine-Inch-Nails',
    
'https://www.musixmatch.com/artist/AC-DC',
    
'https://www.musixmatch.com/artist/The-Rolling-Stones',
    
'https://www.musixmatch.com/artist/Public-Enemy',
    
'https://www.musixmatch.com/artist/Alice-In-Chains',
    
'https://www.musixmatch.com/artist/Cradle-of-Filth',

'https://www.musixmatch.com/artist/Metallica']

In [10]:
bulk_df = []

In [13]:
artist[6:9]

['https://www.musixmatch.com/artist/The-Rolling-Stones',
 'https://www.musixmatch.com/artist/Public-Enemy',
 'https://www.musixmatch.com/artist/Alice-In-Chains']

In [15]:
for i in artist[9:]:
    
    lyric_scrape = LyricScraper(artist_url=i)
    
    df = lyric_scrape.Run(target=60)
    
    bulk_df.append(df)
    

In [16]:
x = pd.concat([i for i in bulk_df])

In [17]:
x

,artist,song,lyrics
0,Kylie-Minogue,Better-the-Devil-You-Know,Better the devil you know Say you won't leave ...
2,Kylie-Minogue,Step-Back-in-Time-Mousse-T-s-Classic-Disco-Shi...,Non stop dancing the bus stop To the funky mus...
3,Kylie-Minogue,Kids,Me no bubbletious Me smoke heavy tar Me be gro...
4,Kylie-Minogue,Timebomb-1,"Fast, time is ticking oh so fast The sound is ..."
5,Kylie-Minogue,Dancing-1,No one wants to stay at home Nobody wants to b...
...,...,...,...
49,Metallica,Fuel-1,"Gimme fuel, gimme fire Gimme that which I desi..."
50,Metallica,Where-the-Wild-Things-Are,So wake up sleepy one It's time to save your w...
51,Metallica,Creeping-Death-1,Slaves! Hebrews born to serve To the pharaoh H...
52,Metallica,All-Nightmare-Long-1,"One, two Luck runs out Crawl from the wreckage..."


In [18]:
x.to_csv('extra.csv',index=False)

In [33]:
df

,artist,song,lyrics
0,Coldplay,The-Scientist,"Come up to meet you, tell you I'm sorry You do..."
1,Coldplay,O,A flock of birds Hovering above Just a flock o...
2,Coldplay,Ink,"Got a tattoo said ""together thru a life"" Carve..."
3,Coldplay,Charlie-Brown,Stole a key Took a car downtown where the lost...
4,Coldplay,Orphans-1,"Rosaleem of the Damascene Yes, she had eyes li..."
5,Coldplay,Yellow,Look at the stars Look how they shine for you ...
6,Coldplay,Champion-Of-The-World-1,I tried my best to be just like The other boys...
7,Coldplay,Birds,Been standing in the corner Studying the light...
8,Coldplay,Warning-Sign,A warning sign I missed the good part then I r...
9,Coldplay,A-Head-Full-of-Dreams,"Oh, I think I've landed In a world I hadn't se..."
